<h1>Задание</h1>

<h5>2.	Используя один из подходящих игрушечных датасетов (breast_canser, digits, diabetes …) применить алгоритмы улучшения качества и оценки моделей (перекрестная проверка, решетчатый поиск, метрики модели)</h5>

In [1]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def load_and_split_data(test_size=0.2, random_state=42):
    """Загружает и разделяет данные на обучающую и тестовую выборки."""
    data = load_diabetes()
    X, y = data.data, data.target
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

def perform_cross_validation(model, X_train, y_train, cv=5):
    """Выполняет перекрёстную проверку и возвращает средний MSE."""
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
    mean_mse = -np.mean(cv_scores)
    print("Средний MSE на перекрёстной проверке:", mean_mse)
    return mean_mse

def perform_grid_search(model, param_grid, X_train, y_train, cv=5):
    """Выполняет поиск гиперпараметров с использованием решетчатого поиска."""
    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    print("Лучшие параметры:", grid_search.best_params_)
    return grid_search.best_estimator_

def evaluate_model(model, X_test, y_test):
    """Оценивает модель на тестовых данных и возвращает MSE."""
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("MSE на тестовых данных:", mse)
    return mse

if __name__ == "__main__":
    # Шаг 1: Загрузка и разделение данных
    X_train, X_test, y_train, y_test = load_and_split_data()

    # Шаг 2: Инициализация модели
    model = RandomForestRegressor(random_state=42)

    # Шаг 3: Перекрёстная проверка
    perform_cross_validation(model, X_train, y_train)

    # Шаг 4: Решетчатый поиск гиперпараметров
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    }
    best_model = perform_grid_search(model, param_grid, X_train, y_train)

    # Шаг 5: Оценка на тестовых данных
    evaluate_model(best_model, X_test, y_test)


Средний MSE на перекрёстной проверке: 3511.9697030744474
Лучшие параметры: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 150}
MSE на тестовых данных: 2957.7682934471345


<h5>3. Создать модель классификатор, которая по определенному обучаемому набору (salary, city, age, vacation_prefer, transport_prefer) будет определять предпочтения человека (target), в каком городе провести отпуск.</h5>
<ol>
    <li><strong>Создать свой датасет с категориальными данными (1000 строк), которые содержат следующие поля:</strong> (salary, city, age, vacation_prefer, transport_prefer, target). Применить <code>import random</code> (<code>random.choice</code>, <code>random.randint</code>).</li>
    <ul>
        <li><strong>Salary:</strong> установить числовой тип (например, 50000).</li>
        <li><strong>City:</strong> город проживания (например, Bishkek).</li>
        <li><strong>Age:</strong> возраст (например, от 30 до 65).</li>
        <li><strong>Vacation_prefer:</strong> тип отдыха (например, Shopping или Beach holiday).</li>
        <li><strong>Transport_prefer:</strong> тип транспорта (например, auto, plane).</li>
        <li><strong>Target:</strong> город, в котором проведем отпуск (например, London, Moscow).</li>
    </ul>
    <li><strong>Преобразовать категориальные данные в числовые, используя</strong> <code>pandas.get_dummies</code>.</li>
    <li><strong>Выделить обучающую выборку и тестовую выборку:</strong> (<code>X_train</code>, <code>y_train</code>, <code>X_test</code>, <code>y_test</code>).</li>
    <li><strong>Выбрать модель классификатор:</strong> (например, <code>from sklearn.ensemble import RandomForestClassifier</code>).</li>
    <li><strong>Проверить оценку модели.</strong></li>
    <li><strong>Сделать предсказание на случайных данных.</strong></li>
    <li><strong>Улучшить модель:</strong> согласно примерам лекций 9–10 и материала лабораторной работы.</li>
</ol>


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Загрузка новой базы данных
file_path = "vacation_preferences_dataset_generated.xlsx"
df = pd.read_excel(file_path)

# Пример данных
print("Пример данных:")
print(df.head())

# Преобразование категориальных данных в числовые
df_encoded = pd.get_dummies(df, columns=['city', 'vacation_prefer', 'transport_prefer', 'target'])

# Определение X (признаки) и y (целевая переменная)
X = df_encoded.drop(columns=[col for col in df_encoded.columns if col.startswith('target_')])
y = df_encoded[[col for col in df_encoded.columns if col.startswith('target_')]].values.argmax(axis=1)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Оценка модели
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точность модели:", accuracy)
print("Отчёт классификации:")
print(classification_report(y_test, y_pred))

# Предсказание на случайных данных
random_data = {
    'salary': [50000],
    'age': [35],
    'city_Bishkek': [1], 'city_Almaty': [0], 'city_Moscow': [0], 'city_London': [0], 'city_Paris': [0],
    'vacation_prefer_Beach holiday': [1], 'vacation_prefer_Cruise': [0],
    'vacation_prefer_Hiking': [0], 'vacation_prefer_Shopping': [0],
    'transport_prefer_auto': [1], 'transport_prefer_plane': [0], 'transport_prefer_train': [0]
}
random_df = pd.DataFrame(random_data)
random_df = random_df.reindex(columns=X_train.columns, fill_value=0)
prediction = model.predict(random_df)
predicted_target = [col.replace('target_', '') for col in df_encoded.columns if col.startswith('target_')][prediction[0]]
print("Предсказание для случайных данных (результат):", predicted_target)

# Улучшение модели
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Лучшая модель
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print("Точность улучшенной модели:", accuracy_best)
print("Лучшие параметры модели:", grid_search.best_params_)


Пример данных:
   salary  age    city vacation_prefer transport_prefer target
0   23316   29   Paris   Beach holiday            plane    Yes
1   47743   58  London          Hiking             auto    Yes
2   34716   49  Moscow        Shopping            plane    Yes
3   89433   47  Almaty          Hiking            train    Yes
4   68620   64  Almaty          Cruise             auto     No
Точность модели: 0.45
Отчёт классификации:
              precision    recall  f1-score   support

           0       0.40      0.42      0.41        45
           1       0.50      0.47      0.49        55

    accuracy                           0.45       100
   macro avg       0.45      0.45      0.45       100
weighted avg       0.45      0.45      0.45       100

Предсказание для случайных данных (результат): No
Точность улучшенной модели: 0.51
Лучшие параметры модели: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 150}


<h1>Причины, по которым улучшение модели оказалось невозможным:</h1>


<h5>Ограниченность данных Датасет имеет ограниченное количество наблюдений или недостаточное разнообразие данных, что не позволяет модели выявить устойчивые закономерности. При недостаточном объеме данных даже оптимизация гиперпараметров не дает значительного прироста точности.
Недостаточная связь признаков с целевой переменной
Признаки (например, salary, age, закодированные категориальные данные) могут быть недостаточно информативными или иметь слабую корреляцию с целевой переменной. Это означает, что модель просто не может найти значимых закономерностей, необходимых для точного предсказания.

Случайный характер данных
Если целевая переменная зависит от факторов, не представленных в данных, даже самая мощная модель не сможет повысить точность. Это указывает на фундаментальные ограничения в исходной информации.

Лимит возможностей алгоритма
Random Forest — мощный алгоритм, но даже он имеет ограничения. Если данные содержат очень сложные, скрытые или нелинейные зависимости, алгоритм может не справиться с их выявлением, особенно если признаки плохо отражают природу задачи.

Проблемы с дисбалансом классов
Если классы целевой переменной представлены неравномерно, даже корректировка class_weight не всегда помогает улучшить метрики. В редких случаях это приводит к тому, что модель плохо обучается на менее распространенных классах.

Выбор гиперпараметров
Несмотря на использование GridSearchCV, диапазон гиперпараметров может быть недостаточно широким, чтобы найти оптимальную конфигурацию. Однако расширение диапазона часто приводит к переобучению при малом объеме данных.

Сложность задачи
Если задача прогнозирования изначально сложна и связана с множеством факторов, не представленных в датасете, точность модели неизбежно ограничивается.

Вывод:
После тщательной оптимизации гиперпараметров и использования сбалансированных весов классов модель достигла точности 51%. Это указывает на фундаментальные ограничения данных или недостаточную связь признаков с целевой переменной. Для дальнейшего улучшения результатов необходимы более качественные данные, дополнительные признаки или пересмотр постановки задачи.</h5>